# Document Classification Project


In [1]:
# set up libraries 
import pandas as pd
import glob
import os
from bs4 import BeautifulSoup
import requests


In [5]:
# Get data
df = pd.DataFrame(columns=['Text', 'Label'])
main_path = r'C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/'
end_path = ['Analytical_Report', 'Incident_Report', 'Situation_Report']
for end in end_path:
    path = main_path + end
    all_files = glob.glob(os.path.join(path, "*.txt")) 
    # print(all_files[0])
    data = [[f, end] for f in all_files]
    # print(pd.DataFrame(data, columns=['Text', 'Label']))
    df = df.append(pd.DataFrame(data, columns=['Text', 'Label']))
print(df)



                                                 Text              Label
0   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
1   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
2   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
3   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
4   C:/Users/gobli/Documents/AI CORE/Machine-Learn...  Analytical_Report
..                                                ...                ...
65  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
66  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
67  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
68  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report
69  C:/Users/gobli/Documents/AI CORE/Machine-Learn...   Situation_Report

[157 rows x 2 columns]


In [3]:
docs = pd.read_csv("C:/Users/gobli/Documents/AI CORE/Machine-Learning-Algos/Adarga_NLP/Data/Data2.csv")
docs.dropna(inplace=True)
print(docs)

                                                   URL              CLASS
1    https://issafrica.org/iss-today/time-to-tackle...  Analytical Report
3    http://www.understandingwar.org/backgrounder/r...  Analytical Report
5    http://www.understandingwar.org/backgrounder/r...  Analytical Report
7    http://www.understandingwar.org/backgrounder/w...  Analytical Report
9    http://www.understandingwar.org/backgrounder/r...  Analytical Report
..                                                 ...                ...
345  https://freedomhouse.org/article/recent-wins-a...  Analytical Report
347  https://freedomhouse.org/article/ensuring-demo...  Analytical Report
349  https://freedomhouse.org/article/hong-kongs-ac...  Analytical Report
351  https://issafrica.org/impact/sadc-strengthens-...  Analytical Report
353  https://issafrica.org/iss-today/tensions-cloud...  Analytical Report

[177 rows x 2 columns]


In [17]:
# Get txt files from urls
for i in range(len(docs)):
    try:
        r = requests.get(docs.iloc[i][0], timeout=10)
    except:
        print("Problem:", i)
        continue

    if r.ok:
        print("Sucess:", i, docs.iloc[i][0])
        soup = BeautifulSoup(r.content, 'html.parser')
        with open("data/Reports/" + str(i) + ".txt", 'wt') as f:
            text = ""
            for element in soup.select('p'):
                # Reduce potential overfitting by not including parts of the document that repeat themselves regularly and aren't relevant
                # to the actual text
                if  "View/Print Poster" in element.text:
                    continue
                elif element.text == "If you prefer a different language, you may select one from the full list by navigating to the Select Language menu at the top of this site.":
                    continue
                elif element.text == "Rewards for Justice":
                    continue
                elif element.text == "The most important reasons to stop a terrorist are all around you.":
                    continue
                elif element.text == "Stop a terrorist. Save lives.":
                    continue
                elif element.text == "Up to $10 Million Reward":
                    continue
                elif element.text == "{{region_detail.text}}":
                    continue
                elif element.text == "{{person_detail.text}}":
                    continue
                elif element.text == "{{organization_detail.text}}":
                    continue
                elif element.text == "{{series.description | limitTo:140}}":
                    continue
                elif element.text == "Up to $5 Million Reward":
                    continue
                elif len(element.text.split()) == 1:
                    continue
                text += element.text + "\n"
            try:
                f.write(text)
            except:
                print("Write problem:", i)
                continue
        


                                                   URL              CLASS
1    https://issafrica.org/iss-today/time-to-tackle...  Analytical Report
3    http://www.understandingwar.org/backgrounder/r...  Analytical Report
5    http://www.understandingwar.org/backgrounder/r...  Analytical Report
7    http://www.understandingwar.org/backgrounder/w...  Analytical Report
9    http://www.understandingwar.org/backgrounder/r...  Analytical Report
..                                                 ...                ...
345  https://freedomhouse.org/article/recent-wins-a...  Analytical Report
347  https://freedomhouse.org/article/ensuring-demo...  Analytical Report
349  https://freedomhouse.org/article/hong-kongs-ac...  Analytical Report
351  https://issafrica.org/impact/sadc-strengthens-...  Analytical Report
353  https://issafrica.org/iss-today/tensions-cloud...  Analytical Report

[177 rows x 2 columns]
Sucess: 0 https://issafrica.org/iss-today/time-to-tackle-violence-against-women-in-mali


In [6]:
# 3, 15, 28, 32, 33, 119

print(len(docs))

177
